Подгружаем библиотеки питона

In [2]:
import numpy as np
import pandas as pd

Объявим несколько функций: одна из них преобразует значения в колонках, вторая добавляет строки в пропущенные ячейки таблицы. Их всего немного поэтому они не будут нам мешать в предсказании и правильном представлении информации

In [3]:
def to_numeric_with_exceptions(column):
    try:
        return pd.to_numeric(column)
    except ValueError:
        return column

def add_rows(df):
    new_rows = []
    for name, group in df.groupby(['Year', 'Month', 'Day']):
        if len(group) != 8:
            missing_hours = [hour for hour in range(0, 24, 3) if hour not in group['Hour'].values]

            for hour in missing_hours:
                new_row = {col: np.nan for col in df.columns}
                new_row['Year'] = name[0]
                new_row['Month'] = name[1]
                new_row['Day'] = name[2]
                new_row['Hour'] = hour
                new_rows.append(new_row)

    if new_rows:
        new_rows_df = pd.DataFrame(new_rows)
        df = pd.concat([df, new_rows_df], ignore_index=True)
        df.sort_values(by=['Year', 'Month', 'Day', 'Hour'], inplace=True)
        df.reset_index(drop=True, inplace=True)

    return df

Прочитаем нашу базу данных

In [4]:
df = pd.read_csv('DataBase.csv',
                 delimiter=';',
                 names = ['Year', 'Month', 'Day', 'Hour', #date parameters
                          'Temp', 'Pressure', 'Humidity', 'Direction', 'Power', 'Cloudiness', #necessary parameters
                          'Weather', 'LowCl', 'LowPerc', 'Height', 'HighCl1', 'HighCl2', 'Visibility', 'Rainfall'], #unnecessary parameters
                 low_memory=False)

Добавим недостающие строки, заполним обязательные для представления поля предыдущими значениями. Уберем первую нулевую строчку и преобразуем все в численный формат

In [5]:
df=add_rows(df)

columns_to_fill = ['Temp', 'Pressure', 'Humidity', 'Direction', 'Power', 'Cloudiness']
df[columns_to_fill] = df[columns_to_fill].ffill()

df = df[1:len(df)]

for col in df.columns:
    df[col] = to_numeric_with_exceptions(df[col])

print(df.head())

   Year  Month  Day  Hour  Temp  Pressure  Humidity Direction  Power  \
1  2005      2    1     3  -8.6     743.0      89.0         7    2.0   
2  2005      2    1     6  -8.2     742.8      90.0         7    3.0   
3  2005      2    1     9  -8.6     743.5      89.0         7    3.0   
4  2005      2    1    12  -7.1     745.0      85.0         7    3.0   
5  2005      2    1    15  -6.6     746.8      83.0         7    3.0   

   Cloudiness Weather                                              LowCl  \
1       100.0    Снег                                    Слоисто-кучевые   
2       100.0    Снег  Слоистые разорванные или кучевые разорванные о...   
3       100.0    Снег  Слоистые разорванные или кучевые разорванные о...   
4        95.0    Снег                                    Слоисто-кучевые   
5       100.0  Ливень                                    Кучевые средние   

   LowPerc  Height                                          HighCl1  \
1     75.0   800.0                   Вы

Выгрузим нашу базу в файл Temp.csv с разделителем ";"

In [8]:
df.to_csv('Temp.csv', sep=';')